In [1]:
import numpy as np
import pandas as pd 
import matplotlib as plt 
import seaborn as sns
from numba import jit 
import fancyimpute 
import matplotlib.pyplot as plt
%matplotlib inline

/home/sid/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


IMPORTING THE FILES

In [2]:
df_train=pd.read_csv('train.csv')
df_test=pd.read_csv('test.csv')

PREPROCESSING TRAIN DATA

In [5]:
df_train['Age']=df_train.Age.map({'0-17':1,'18-25':2,'26-35':3,'36-45':4,'46-50':4,'51-55':5,'55+':6}).astype(np.float)
df_train['Stay_In_Current_City_Years']=df_train['Stay_In_Current_City_Years'].str.extract('(\d+)').astype(np.float)
df_train['Gender']=df_train.Gender.map({'F':0,'M':1}).astype(np.float)
df_train['City_Category']=df_train.City_Category.map({'A':0,'B':1,'C':2}).astype(np.float)
df_train['Product_ID']=df_train['Product_ID'].str.extract('(\d+)').astype(np.float)
df_train['User_ID']=df_train['User_ID']%1000000
df_train=df_train.fillna(99)

/home/sid/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
  
/home/sid/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
  """


PREPROCESSING TEST DATA

In [6]:
df_test['Age']=df_test.Age.map({'0-17':1,'18-25':2,'26-35':3,'36-45':4,'46-50':4,'51-55':5,'55+':6}).astype(np.float)
df_test['Stay_In_Current_City_Years']=df_test['Stay_In_Current_City_Years'].str.extract('(\d+)').astype(np.float)
df_test['Gender']=df_test.Gender.map({'F':0,'M':1}).astype(np.float)
df_test['City_Category']=df_test.City_Category.map({'A':0,'B':1,'C':2}).astype(np.float)
df_test['Product_ID']=df_test['Product_ID'].str.extract('(\d+)').astype(np.float)
df_test['User_ID']=df_test['User_ID']%1000000
df_test=df_test.fillna(99)

/home/sid/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
  
/home/sid/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
  """


STACKING THE TWO TRAIN AND TEST TOGETHER

In [7]:
df_full=pd.concat([df_train,df_test],axis=0)

CHECKING FOR UNIQUE PRODUCT_ID 

In [8]:
lm=pd.unique(df_train['Product_ID'])
r=lm.shape
r=r[0]


In [9]:
df_mean=df_full.Purchase.mean()

In [10]:
df_full['Pvalue']=0


THE BELOW FOR LOOP DOES THE FOLLOWING :
1. CREATS A NEW Ith column
2. VAL = The mean of purchase value according to the unique product id
3. Mapping val to the ith column according to product id which is in lm array
4. Since in map funtion the other values becomes NaN hence filling it with 0
5. Now adding Pvalue created above to the Ith coloumn 
6. Finally droping the Ith coloumn so that memory does not exceeds 

Repeating this untill all the unique values of product_id

In [11]:
%%time
for i in range (0,r):
    df_full[i]=0
    val=df_full['Purchase'].where(df_full['Product_ID']==lm[i],axis=0).mean()
    df_full[i]=df_full[i].map({0:val}).where(df_full['Product_ID']==lm[i],axis=0)
    df_full=df_full.fillna(0)
    df_full['Pvalue']=df_full['Pvalue']+df_full[i]
    df_full=df_full.drop([i],axis=1)

CPU times: user 9min 38s, sys: 4min 6s, total: 13min 44s
Wall time: 12min 14s


Now as we can see our Pvalue is ready but still their are some product ID that  did not have any purchase , those are being imputed 
by the mean of purchase , as this data set is very big few random changes will no have an issue

In [13]:
 def change(val):
        if val==0:
            return df_mean
df_full['Pvalue1']=df_full['Pvalue'].apply(change)
df_full=df_full.fillna(0) #this is important becouse if we ad NA to another coloumn , that column replaces its value with NA
df_full['Pvalue']=df_full['Pvalue1']+df_full['Pvalue']
df_full=df_full.drop(['Pvalue1'],axis=1)

Now creating Buying power per user-Id

In [14]:
lm=pd.unique(df_full['User_ID'])
r=lm.shape
r=r[0]


In [15]:
df_full['Bpower']=0

THE BELOW FOR LOOP DOES THE FOLLOWING :

1.   CREATS A NEW Ith column
2.   VAL = The mean of purchase value according to the unique USER_ID
3.   Mapping val to the ith column according to USER_ID which is in lm array
4.   Since in map funtion the other values becomes NaN hence filling it with 0
5.   Now adding Pvalue created above to the Ith coloumn
6.   Finally droping the Ith coloumn so that memory does not exceeds

Repeating this untill all the unique values of product_id


In [16]:
for i in range (0,r):
    df_full[i]=0
    val=df_full['Purchase'].where(df_full['User_ID']==lm[i],axis=0).mean()
    df_full[i]=df_full[i].map({0:val}).where(df_full['User_ID']==lm[i],axis=0)
    df_full=df_full.fillna(0)
    df_full['Bpower']=df_full['Bpower']+df_full[i]
    df_full=df_full.drop([i],axis=1)

In [17]:
#plotting correlation matrix 
#Hence we are keeping the full dataset intact 

df_full.corr(method='pearson').style.format("{:.2}").background_gradient(cmap=plt.get_cmap('coolwarm'), axis=1)

,Age,City_Category,Gender,Marital_Status,Occupation,Product_Category_1,Product_Category_2,Product_Category_3,Product_ID,Purchase,Stay_In_Current_City_Years,User_ID,Pvalue,Bpower
Age,1.0,0.11,0.0021,0.29,0.096,0.059,0.019,0.021,0.028,0.0098,0.0027,0.037,0.016,0.048
City_Category,0.11,1.0,-0.0037,0.04,0.035,-0.018,-0.034,-0.039,0.0086,0.037,0.021,0.022,0.044,0.18
Gender,0.0021,-0.0037,1.0,-0.011,0.12,-0.046,-0.014,-0.032,0.021,0.036,0.015,-0.035,0.071,0.18
Marital_Status,0.29,0.04,-0.011,1.0,0.024,0.02,0.0099,0.01,0.013,-0.00055,-0.013,0.02,0.0031,-0.0027
Occupation,0.096,0.035,0.12,0.024,1.0,-0.0084,-0.011,-0.011,0.0085,0.012,0.028,-0.024,0.02,0.058
Product_Category_1,0.059,-0.018,-0.046,0.02,-0.0084,1.0,0.47,0.45,0.076,-0.2,-0.0035,0.0046,-0.39,-0.11
Product_Category_2,0.019,-0.034,-0.014,0.0099,-0.011,0.47,1.0,0.49,0.14,-0.15,-0.0037,-0.0032,-0.29,-0.1
Product_Category_3,0.021,-0.039,-0.032,0.01,-0.011,0.45,0.49,1.0,0.16,-0.19,-0.0011,-0.0019,-0.37,-0.11
Product_ID,0.028,0.0086,0.021,0.013,0.0085,0.076,0.14,0.16,1.0,-0.057,-0.0033,-0.018,-0.11,-0.014
Purchase,0.0098,0.037,0.036,-0.00055,0.012,-0.2,-0.15,-0.19,-0.057,1.0,0.0033,0.0027,0.5,0.2


Here there is significant corelation between Purchase and the two newly created features Pvalue and Bpower

In [18]:
lm=pd.unique(df_full['Pvalue'])
r=lm.shape
r=r[0]
r


3626

In [19]:
km=pd.unique(df_full['Product_ID'])
m=km.shape
m=m[0]
m


3677

In [20]:
df_full['Pmax']=0

Here we are Adding another feature which will count the number of times that product have been bought at price above its actual mean value the process is same as that of previous loops

In [21]:
%%time
for i in range (0,r):
    df_full[i]=0
    val=df_full['Product_ID'].where(df_full['Purchase']>lm[i]).where(df_full['Product_ID']==km[i],axis=0).count()
    df_full[i]=df_full[i].map({0:val}).where(df_full['Product_ID']==km[i],axis=0)
    df_full=df_full.fillna(0)
    df_full['Pmax']=df_full['Pmax']+df_full[i]
    df_full=df_full.drop([i],axis=1)

CPU times: user 19min 15s, sys: 7min 15s, total: 26min 30s
Wall time: 24min 9s


In [22]:
df_full['Pmax1']=0

In [23]:
df_full['Pmax1']=df_full['Pmax1'].map({0:298}).where(df_full['Pmax']==0,axis=0)
df_full=df_full.fillna(0)
df_full['Pmax']=df_full['Pmax']+df_full['Pmax1']
df_full=df_full.drop('Pmax1',axis=1)


In [24]:
df_xtrain=df_full.iloc[:550068,:]
df_xtest=df_full.iloc[550068:,:]
df_xtest=df_xtest.drop(['Purchase'],axis=1)

In [25]:
#plotting correlation matrix 
#Hence we are keeping the full dataset intact 

df_xtrain.corr(method='pearson').style.format("{:.2}").background_gradient(cmap=plt.get_cmap('coolwarm'), axis=1)

,Age,City_Category,Gender,Marital_Status,Occupation,Product_Category_1,Product_Category_2,Product_Category_3,Product_ID,Purchase,Stay_In_Current_City_Years,User_ID,Pvalue,Bpower,Pmax
Age,1.0,0.11,0.0028,0.29,0.096,0.059,0.021,0.021,0.029,0.017,0.0027,0.037,0.014,0.049,-0.013
City_Category,0.11,1.0,-0.0045,0.04,0.034,-0.014,-0.032,-0.039,0.01,0.062,0.02,0.023,0.041,0.19,0.1
Gender,0.0028,-0.0045,1.0,-0.012,0.12,-0.046,-0.014,-0.032,0.021,0.06,0.015,-0.033,0.07,0.18,0.019
Marital_Status,0.29,0.04,-0.012,1.0,0.024,0.02,0.011,0.0097,0.013,-0.00046,-0.013,0.02,0.0035,-0.0033,-0.0079
Occupation,0.096,0.034,0.12,0.024,1.0,-0.0076,-0.0094,-0.0099,0.0093,0.021,0.03,-0.024,0.02,0.059,0.011
Product_Category_1,0.059,-0.014,-0.046,0.02,-0.0076,1.0,0.47,0.45,0.09,-0.34,-0.0042,0.0038,-0.4,-0.11,-0.23
Product_Category_2,0.021,-0.032,-0.014,0.011,-0.0094,0.47,1.0,0.49,0.15,-0.25,-0.0031,-0.004,-0.29,-0.1,-0.26
Product_Category_3,0.021,-0.039,-0.032,0.0097,-0.0099,0.45,0.49,1.0,0.16,-0.31,-0.0016,-0.003,-0.37,-0.12,-0.29
Product_ID,0.029,0.01,0.021,0.013,0.0093,0.09,0.15,0.16,1.0,-0.1,-0.0032,-0.018,-0.12,-0.013,-0.18
Purchase,0.017,0.062,0.06,-0.00046,0.021,-0.34,-0.25,-0.31,-0.1,1.0,0.0054,0.0047,0.85,0.31,0.33


As we can see these new features have significant correlation with the purchase amount

Now we will Cross validate and check for RMSE Score 


In [27]:
y=df_xtrain.Purchase
x=df_xtrain.drop(['Purchase'],axis=1)


In [28]:
from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 0)

/home/sid/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [29]:
import sys
import numpy as np
import pandas as pd
import xgboost as xgb


In [36]:
# Modeling
dtrain = xgb.DMatrix(X_train, label=y_train)# here train.values is training on dataset

param = {'objective': 'reg:linear', 'booster': 'gbtree', 'silent': 1, 
		 'max_depth': 10, 'eta': 0.1, 'nthread': 7,# PLEASE CHANGE THIS PARAMETER ACCORDING TO YOUR NUMBER OF CORES REQUIRED FOR COMPUTING
		 'subsample': 0.8, 'colsample_bytree': 0.8, 'min_child_weight': 20,
		 'max_delta_step': 0, 'gamma': 0}
num_round = 1000

# here objective denotes the reg:linear that is whther regression or classification.
# num_round = number of trees we want.
#

In [37]:

seeds = [1122, 2244, 3366, 4488, 5500,6500,7500,9000,10000,15000,20000,25000,35000,40000,45000,50000]
test_preds = np.zeros((len(X_test), len(seeds)))


In [38]:
%%time # This will require a 2 hrs  with 7 core @ 100% 
for run in range(len(seeds)):
	sys.stdout.write("\rXGB RUN:{}/{}".format(run+1, len(seeds)))
	sys.stdout.flush()
	param['seed'] = seeds[run]
	clf = xgb.train(param, dtrain, num_round)
	dtest = xgb.DMatrix(X_test)# Here the test data is given 
	test_preds[:, run] = clf.predict(dtest)

test_preds = np.mean(test_preds, axis=1)


XGB RUN:16/16CPU times: user 7h 38min 12s, sys: 8min 3s, total: 7h 46min 16s
Wall time: 1h 7min 23s


In [39]:
from sklearn.metrics import mean_squared_error
mean_squared_error(y_test,test_preds)**(.5)

2431.3779943351765

THIS IS THE RMSE VALUE WHICH WILL DEVIATE BY +10 WHILE USING THE TEST 

PLEASE CHANGE THE VALUES OF TEST AND TRAIN DATA SET BEFORE FINAL SUBMISSION 
BELOW IS THE SUBMISSION CODE 


In [ ]:
df_test1=pd.read_csv('test.csv')
submission = pd.DataFrame({
        
        "User_ID": df_test1["User_ID"],
        "Product_ID":df_test1["Product_ID"],
        "Purchase":test_preds
        } )
cols=["User_ID","Product_ID","Purchase"]
submission=submission.reindex(columns=cols)

In [ ]:
submission.to_csv('xgb.csv')